In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders
from model import get_pretrained_emb, EncoderRNN, DecoderRNN, DecoderAttnRNN, EncoderDecoder, EncoderDecoderAttn
from train_eval import train_and_eval, count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
import pickle as pkl 
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# params required for generating data loaders 

SRC_LANG = 'vi'
TARG_LANG = 'en'

SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 
TARG_VOCAB_SIZE = 30000 

BATCH_SIZE = 64 

In [3]:
# takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [4]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, filter_long=False)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE, filter_long=False) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000, filter_long=False)

In [5]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [6]:
# model architecture params 
NETWORK_TYPE = 'rnn'
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 
DEC_DROPOUT = 0.2 
ATTENTION_TYPE = 'additive'

# training params  
NUM_EPOCHS = 5
LR = 0.0003 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = False

# name the model and experiment 
EXPERIMENT_NAME = 'hyperparameter_tuning_dropout'
if NETWORK_TYPE == 'rnn': 
    MODEL_NAME = '{}-rnn-{}-attn'.format(SRC_LANG, ATTENTION_TYPE)
elif NETWORK_TYPE == 'cnn': 
    MODEL_NAME = '{}-cnn'.format(SRC_LANG)

In [7]:
# store as dict to save to results later 
params = {'experiment_name': EXPERIMENT_NAME,'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 
          'rnn_cell_type': RNN_CELL_TYPE, 'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 
          'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 'src_vocab_size': SRC_VOCAB_SIZE, 
          'targ_vocab_size': TARG_VOCAB_SIZE, 'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 
          'dec_hidden_dim': DEC_HIDDEN_DIM, 'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 
          'clip_grad_max_norm': CLIP_GRAD_MAX_NORM, 'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 
          'attention_type': ATTENTION_TYPE, 'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 
          'learning_rate': LR, 'optimizer': OPTIMIZER, 'lazy_train': LAZY_TRAIN} 

In [ ]:
for candidate in [0, 0.2, 0.5]: 
    
    print("Training with dropout = {}".format(candidate))
    
    # overwrite relevant key-value in params 
    params['enc_dropout'] = candidate 
    params['dec_dropout'] = candidate
    params['model_name'] = '{}-rnn-{}-attn-{}-dropout'.format(SRC_LANG, ATTENTION_TYPE, candidate)
    
    # instantiate model 
    encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                         src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=candidate, 
                         pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
    decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                             num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, 
                             src_max_sentence_len=SRC_MAX_SENTENCE_LEN, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                             dec_dropout=candidate, attention_type=ATTENTION_TYPE,
                             pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], 
                                                                    vocab[TARG_LANG]['token2id']))
    model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']).to(device) 
    
    # train and eval 
    model, results = train_and_eval(
        model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
        params=params, vocab=vocab, print_intermediate=100, save_checkpoint=True, save_to_log=True, 
        lazy_eval=True, print_attn=True, inspect_samples=1)

Training with dropout = 0
Epoch: 0.00, Train Loss: 0.00, Val Loss: 10.23, Train BLEU: 0.00, Val BLEU: 0.07, Minutes Elapsed: 0.12
Sampling from val predictions...
Source: chúng_tôi không_thể chỉnh sửa nếu ảnh không được làm sạch
Reference: we couldn &apos;t <UNK> the photo unless it was
Model: <SOS> and the the the to to to to to
Attention Weights: tensor([[0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002, 0.1009, 0.1012, 0.1016,
         0.1023],
        [0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002, 0.1009, 0.1012, 0.1016,
         0.1023],
        [0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002, 0.1009, 0.1012, 0.1016,
         0.1023],
        [0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002, 0.1009, 0.1012, 0.1016,
         0.1023],
        [0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002, 0.1009, 0.1012, 0.1016,
         0.1023],
        [0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002, 0.1009, 0.1012, 0.1016,
         0.1023],
        [0.0981, 0.0985, 0.0990, 0.0989, 0.0993, 0.1002

Epoch: 0.05, Train Loss: 0.00, Val Loss: 6.41, Train BLEU: 0.00, Val BLEU: 0.03, Minutes Elapsed: 2.02
Sampling from val predictions...
Source: khi trượt tay , tôi <UNK> nhớ lại người thợ_mỏ
Reference: when my hand slips , i suddenly remember a
Model: <SOS> and , , , , , , , ,
Attention Weights: tensor([[5.1922e-05, 1.5019e-02, 5.3390e-02, 8.7278e-02, 1.1048e-01, 1.3662e-01,
         1.8327e-01, 2.1258e-01, 1.7476e-01, 2.6558e-02],
        [5.8412e-06, 2.9416e-03, 2.0450e-02, 4.7121e-02, 7.3414e-02, 1.1118e-01,
         1.9768e-01, 2.8167e-01, 2.4148e-01, 2.4065e-02],
        [1.4398e-05, 3.4683e-03, 2.1164e-02, 4.7087e-02, 7.2829e-02, 1.1000e-01,
         1.9580e-01, 2.8066e-01, 2.4268e-01, 2.6297e-02],
        [1.4583e-05, 3.4601e-03, 2.1071e-02, 4.6859e-02, 7.2455e-02, 1.0947e-01,
         1.9522e-01, 2.8066e-01, 2.4398e-01, 2.6803e-02],
        [1.4359e-05, 3.4351e-03, 2.0976e-02, 4.6701e-02, 7.2252e-02, 1.0922e-01,
         1.9500e-01, 2.8073e-01, 2.4466e-01, 2.7019e-02],
        

Epoch: 0.14, Train Loss: 0.00, Val Loss: 6.23, Train BLEU: 0.00, Val BLEU: 0.61, Minutes Elapsed: 5.44
Sampling from val predictions...
Source: đó là cách duy_nhất để chúng_tôi có_thể đi học .
Reference: it was the only way we both could be
Model: <SOS> and , , , , , , , ,
Attention Weights: tensor([[0.2524, 0.2080, 0.1439, 0.0849, 0.0822, 0.0634, 0.0560, 0.0483, 0.0353,
         0.0257],
        [0.1252, 0.1335, 0.1301, 0.1127, 0.1108, 0.1020, 0.0946, 0.0833, 0.0634,
         0.0444],
        [0.0911, 0.1008, 0.1038, 0.1047, 0.1049, 0.1051, 0.1050, 0.1034, 0.0982,
         0.0831],
        [0.0783, 0.0952, 0.1017, 0.1050, 0.1051, 0.1063, 0.1066, 0.1056, 0.1028,
         0.0933],
        [0.0700, 0.0928, 0.1018, 0.1060, 0.1059, 0.1075, 0.1078, 0.1070, 0.1046,
         0.0966],
        [0.0621, 0.0906, 0.1021, 0.1073, 0.1070, 0.1089, 0.1092, 0.1084, 0.1062,
         0.0982],
        [0.0546, 0.0882, 0.1024, 0.1087, 0.1083, 0.1104, 0.1107, 0.1098, 0.1076,
         0.0993],
        [0.047

Epoch: 0.19, Train Loss: 0.00, Val Loss: 6.15, Train BLEU: 0.00, Val BLEU: 0.24, Minutes Elapsed: 7.13
Sampling from val predictions...
Source: công_nghiệp dệt may cũng thường được nghĩ đến khi nghe
Reference: the textile industry is another one we often think
Model: <SOS> so we we , , , , , ,
Attention Weights: tensor([[0.9637, 0.0306, 0.0025, 0.0011, 0.0007, 0.0005, 0.0004, 0.0003, 0.0002,
         0.0001],
        [0.3676, 0.1829, 0.0980, 0.0751, 0.0644, 0.0569, 0.0511, 0.0453, 0.0353,
         0.0233],
        [0.2265, 0.1508, 0.1039, 0.0916, 0.0853, 0.0815, 0.0765, 0.0729, 0.0632,
         0.0478],
        [0.1219, 0.1354, 0.1121, 0.1032, 0.0988, 0.0965, 0.0930, 0.0899, 0.0824,
         0.0667],
        [0.0775, 0.1231, 0.1152, 0.1090, 0.1061, 0.1043, 0.1020, 0.0983, 0.0910,
         0.0734],
        [0.0480, 0.1069, 0.1165, 0.1139, 0.1125, 0.1114, 0.1101, 0.1055, 0.0977,
         0.0775],
        [0.0303, 0.0903, 0.1162, 0.1174, 0.1178, 0.1172, 0.1168, 0.1111, 0.1030,
         0.

Epoch: 0.24, Train Loss: 0.00, Val Loss: 6.00, Train BLEU: 0.00, Val BLEU: 2.70, Minutes Elapsed: 8.84
Sampling from val predictions...
Source: tại tây phi có_một ngành buôn_bán kền_kền <UNK> kinh_khủng để
Reference: in west africa , there &apos;s a horrific trade
Model: <SOS> it &apos;s a a a , , , ,
Attention Weights: tensor([[9.8392e-01, 1.5370e-02, 5.1550e-04, 1.0617e-04, 3.7389e-05, 1.6875e-05,
         9.7165e-06, 8.6830e-06, 6.4099e-06, 4.6741e-06],
        [2.4828e-01, 2.2104e-01, 1.7124e-01, 1.3725e-01, 7.9021e-02, 4.4680e-02,
         3.0712e-02, 2.9121e-02, 2.1486e-02, 1.7167e-02],
        [6.3238e-02, 9.8611e-02, 1.3943e-01, 1.3483e-01, 1.1997e-01, 1.0765e-01,
         9.4088e-02, 8.4412e-02, 8.3641e-02, 7.4136e-02],
        [6.6530e-03, 1.4940e-02, 5.4998e-02, 7.4857e-02, 1.6067e-01, 1.7166e-01,
         1.4833e-01, 9.6149e-02, 1.4472e-01, 1.2703e-01],
        [3.1114e-03, 7.8160e-03, 5.2068e-02, 7.4738e-02, 1.7170e-01, 1.7981e-01,
         1.4923e-01, 9.1983e-02, 1.4527e-

...

Epoch: 1.48, Train Loss: 0.00, Val Loss: 4.44, Train BLEU: 0.00, Val BLEU: 11.02, Minutes Elapsed: 60.12
Sampling from val predictions...
Source: tuy_nhiên , sau khi chúng_tôi qua được biên_giới , gia_đình
Reference: but even after we got past the border ,
Model: <SOS> however , , when we when , , ,
Attention Weights: tensor([[9.9859e-01, 1.4093e-03, 8.0289e-07, 2.0302e-08, 1.1705e-09, 2.5788e-10,
         5.5442e-11, 2.9751e-11, 5.9946e-12, 3.4534e-12],
        [4.8279e-01, 4.4383e-01, 6.3945e-02, 6.1303e-03, 1.7545e-03, 7.4494e-04,
         3.8457e-04, 2.4313e-04, 9.4576e-05, 8.0722e-05],
        [2.2330e-01, 3.3036e-01, 3.3713e-01, 9.2573e-02, 8.7723e-03, 4.5201e-03,
         2.0207e-03, 8.1031e-04, 2.6007e-04, 2.6393e-04],
        [6.5365e-03, 4.9148e-02, 3.5103e-01, 4.2441e-01, 1.2442e-01, 2.6643e-02,
         9.8913e-03, 3.8870e-03, 1.8439e-03, 2.1818e-03],
        [1.8729e-03, 4.3023e-03, 3.0964e-02, 2.1997e-01, 3.5890e-01, 2.9077e-01,
         6.4104e-02, 1.7136e-02, 6.0290e-03

Epoch: 1.53, Train Loss: 0.00, Val Loss: 4.45, Train BLEU: 0.00, Val BLEU: 11.29, Minutes Elapsed: 62.62
Sampling from val predictions...
Source: tôi đã bị sốc . <EOS> <PAD> <PAD> <PAD> <PAD>
Reference: i was so shocked . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> i was still . . <EOS> . <EOS> <EOS>
Attention Weights: tensor([[0.9819, 0.0170, 0.0010, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0071, 0.3391, 0.6224, 0.0306, 0.0007, 0.0001, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0031, 0.0802, 0.6689, 0.2405, 0.0069, 0.0005, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0006, 0.0041, 0.1314, 0.7947, 0.0413, 0.0280, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0016, 0.0065, 0.1165, 0.3631, 0.1962, 0.3160, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0031, 0.0489, 0.2273, 0.2104, 0.0810, 0.4293, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0115, 0.0903, 0.3893, 0.2270, 0.0564, 0.2255, 0.0000, 0.0000, 0.0000,
         0.

Epoch: 1.58, Train Loss: 0.00, Val Loss: 4.40, Train BLEU: 0.00, Val BLEU: 11.69, Minutes Elapsed: 65.07
Sampling from val predictions...
Source: tất_cả những điều đó hoàn_toàn có_thể chỉ với chiếc điện_thoại_di_động
Reference: all this is possible with your mobile phone .
Model: <SOS> all of that all to to to to to
Attention Weights: tensor([[4.6944e-01, 3.7211e-01, 1.5693e-01, 1.5048e-03, 2.0223e-05, 3.2773e-07,
         3.1045e-08, 6.1928e-09, 2.2108e-09, 1.3880e-09],
        [6.7837e-02, 5.1354e-01, 3.6263e-01, 3.9730e-02, 1.5146e-02, 5.7818e-04,
         3.5798e-04, 1.0559e-04, 4.3536e-05, 2.9426e-05],
        [3.2121e-02, 3.8652e-01, 2.1117e-01, 1.3108e-01, 2.2063e-01, 1.1209e-02,
         5.2995e-03, 1.3128e-03, 4.0486e-04, 2.4301e-04],
        [3.9306e-03, 4.7715e-02, 3.3299e-02, 9.1246e-02, 6.5966e-01, 6.6258e-02,
         8.4978e-02, 1.0015e-02, 1.9685e-03, 9.3447e-04],
        [6.7521e-04, 7.0094e-03, 9.9645e-03, 2.1527e-02, 4.5303e-01, 9.1135e-02,
         2.8971e-01, 1.074

Epoch: 1.62, Train Loss: 0.00, Val Loss: 4.36, Train BLEU: 0.00, Val BLEU: 12.22, Minutes Elapsed: 67.55
Sampling from val predictions...
Source: không có đạo_diễn nghệ_thuật , không có nhà tạo mẫu
Reference: there were no art directors , no <UNK> ,
Model: <SOS> there &apos;s no , , no no &apos;s no
Attention Weights: tensor([[9.9924e-01, 7.3552e-04, 2.7496e-05, 1.5152e-06, 1.4876e-07, 8.1551e-08,
         3.4223e-08, 1.8255e-08, 1.0682e-08, 1.1487e-08],
        [4.0242e-01, 1.9920e-01, 3.2816e-01, 6.3303e-02, 1.9295e-03, 2.3042e-03,
         7.6198e-04, 8.4073e-04, 5.9900e-04, 4.7696e-04],
        [2.5574e-01, 1.0452e-01, 4.4169e-01, 1.6885e-01, 9.7285e-03, 1.2906e-02,
         2.8376e-03, 1.7940e-03, 1.1116e-03, 8.2146e-04],
        [2.1976e-02, 3.3590e-02, 6.4399e-01, 2.7184e-01, 1.2712e-02, 8.4235e-03,
         3.1707e-03, 2.1536e-03, 1.2227e-03, 9.2028e-04],
        [3.0774e-03, 6.3533e-03, 2.4833e-01, 5.3265e-01, 1.2282e-01, 5.2327e-02,
         7.9131e-03, 1.2418e-02, 7.4897e-03

Epoch: 1.67, Train Loss: 0.00, Val Loss: 4.37, Train BLEU: 0.00, Val BLEU: 11.43, Minutes Elapsed: 70.04
Sampling from val predictions...
Source: và cuối_cùng , tôi đã có một thoả_thuận với họ
Reference: so at the end , i had a settlement
Model: <SOS> and finally , , , i had a a
Attention Weights: tensor([[2.0793e-02, 9.7721e-01, 1.9746e-03, 2.2227e-05, 3.2071e-06, 3.6737e-07,
         1.3984e-08, 1.0584e-09, 5.3013e-09, 1.1298e-09],
        [3.2284e-03, 9.8454e-01, 1.1101e-02, 7.9759e-04, 2.4356e-04, 7.3196e-05,
         1.0422e-05, 3.3624e-06, 2.7502e-06, 1.4081e-06],
        [9.5506e-03, 7.4993e-01, 1.9630e-01, 1.7733e-02, 1.5063e-02, 8.9483e-03,
         1.7433e-03, 3.6707e-04, 2.6143e-04, 1.0971e-04],
        [5.7078e-03, 8.2584e-02, 2.0385e-01, 2.3488e-01, 3.2442e-01, 1.2894e-01,
         1.3861e-02, 2.7580e-03, 1.9197e-03, 1.0743e-03],
        [3.1338e-03, 1.8165e-02, 4.7098e-01, 2.4799e-01, 1.5908e-01, 7.3851e-02,
         1.6335e-02, 4.0443e-03, 3.8091e-03, 2.6084e-03],
      

In [ ]:
# experiment_results = load_experiment_log(experiment_name=EXPERIMENT_NAME)
# summarize_results(experiment_results)[['model_name', 'enc_dropout', 'dec_dropout', 'best_val_loss', 'best_val_bleu', 
#                                        'total_params', 'trainable_params', 'runtime']]